# Energy A.I. Hackathon 2021 Workflow - Team Anticline
## Preprocessing
#### Authors: Esmail Eltahan, Jonghyeon Jeon, Mehran Mehrabi, and Wen Pan, Hildebrand Department of Petroleum and Geosystems Engineering. 
#### The University of Texas at Austin, Austin, Texas USA 

<img src="anticline_official_logo.png" width=300 />



## Import Packages

In [1]:
import numpy as np                                          # ndarrys for gridded data
import pandas as pd                                         # DataFrames for tabular data
import matplotlib
import matplotlib.pyplot as plt                             # for plotting
from scipy import spatial
import seaborn as sns       #

## Load the data 



In [2]:
ai = np.load("2d_ai.npy")
sand_propotion = np.load("2d_sand_propotion.npy")
sandy_shale_propotion = np.load("2d_sandy_shale_propotion.npy")
shale_propotion = np.load("2d_shale_propotion.npy")
shaly_sand_propotion = np.load("2d_shaly_sand_propotion.npy")
top_depth = np.load("2d_top_depth.npy")
df_production = pd.read_csv('production_history.csv')
df_production.columns = ['Well_ID','Cumoil1', 'Cumoil2', 'Cumoil3', 'Cumwater1', 'Cumwater2', 'Cumwater3']

woc_rise_1 = np.load("./engineered_maps/woc_rise_1.npy")
woc_rise_2 = np.load("./engineered_maps/woc_rise_2.npy")
woc_rise_3 = np.load("./engineered_maps/woc_rise_3.npy")

## Calculate properties

Here we calculate some properties from the dataset that might (or might not) turn useful. In the following code, we define the following properties at the well locations:

- `Log_perm`: natural logarithm of permeability
- `mindist`: distance to the nearest well
- `log_mindist`: Natural logarithm of distance to the nearest well
- `avg_inv_dist`: sum of inverse distance to neighbouring wells
- `avg_sq_inv_dist`: sum of square inverse distance to neighbouring wells
- `avg_sqrt_inv_dist`: sum of the square root of  inverse distance to neighbouring wells
- `Top_depth`: depth to the top of formation
- `distance to woc`: distance from depth to OWOC
- `location`: whether above or below the fault line
- `distance to fault`: distance to fault
- `fault indicator`: defines proximity to the fault, takes 1 if `distance to fault > 500`, and takes 0 if otherwise
- `Avg_perm`: average permeability from well log measurements over thickness of well
- `thickness`: thickness of sampled interval
- `bottom to woc`: distance from bottom depth of well to water oil contact 
- `sand prp.`: sand samples per all measurements at well location
- `sandshaly prp.`: sum of sand and shaly sand divided by all measurements at well location
- `k/log_mindist`: permeability divided by log of the distance to the nearest well

We derive the following properties from the production data
- `total1`: total production during the first year
- `total2`: total production during the second year 
- `total3`: total production during the third year
- `prod_o1`: oil production during the first year
- `prod_o2`: oil production during the second year 
- `prod_o3`: oil production during the third year
- `prod_w1`: water production during the first year
- `prod_w2`: water production during the second year
- `prod_w3`: water production during the third year 
- `oil12`: change in oil production from year 1 to year 2
- `oil23`: change in oil production from year 2 to year 3
- `water12`: change in water production from year 1 to year 2
- `water23`: change in water production from year 1 to year 2
- `wcut1`: water cut during the first year
- `wcut2`: water cut during the second year
- `wcut3`: water cut during the third year
- `wcutincrease1`: water cut increase from 1st to 2nd year
- `wcutincrease2`: water cut increase from 2nd to 3rd year



In [3]:
def derive_static_prop(df, avg=True):
    woc = 3067.4
    distance_to_woc = woc - top_depth

    xy1 = np.array([df['X'].values, df['Y'].values]).T
    kdtree = spatial.cKDTree(xy1)
    size_arr = xy1.shape[0]
    mindist=np.zeros(size_arr)
    avg_inv_dist=np.zeros(size_arr)
    avg_sq_inv_dist=np.zeros(size_arr)
    avg_sqrt_inv_dist=np.zeros(size_arr)
    df['Log_perm'] = np.log(df['Perm'])

    for i in range(size_arr):
        arr_dist = kdtree.query(xy1[i], k=100)[0][np.nonzero(kdtree.query(xy1[i], k=100)[0])]
        mindist[i] = (arr_dist).min()
        avg_inv_dist[i] = np.sum(1/(20*arr_dist))
        avg_sq_inv_dist[i] = np.sum(1/(20*np.power(arr_dist, 2)))
        avg_sqrt_inv_dist[i] = np.sum(1/(20*np.sqrt(arr_dist)))

    df["mindist"] = mindist
    df["avg_inv_dist"] = avg_inv_dist
    df["avg_sq_inv_dist"] = avg_sq_inv_dist
    df["avg_sqrt_inv_dist"] = avg_sqrt_inv_dist
    df["log_mindist"] = np.log(mindist)
    df["Top_depth"] = top_depth[df['X'].astype('int')//50, df['Y'].astype('int')//50]
    df["interval"] = df.groupby("Well_ID")["Depth"].diff(1)
    df.loc[np.isnan(df["interval"]),"interval"] = df["Depth"] - df["Top_depth"]
    df["distance to woc"] =  woc - df["Depth"]
    unique_list = df["Well_ID"].unique()
    df.loc[df['Y'] >= (-df['X'] + 11750), "location"] = "below fault line"
    df.loc[df['Y'] < (-df['X'] + 11750), "location"] = "above fault line"
    x0y0=np.array([0,11750])
    dxdy=df[['X', 'Y']] .values-x0y0
    dist=((dxdy[:,0]+dxdy[:,1])/np.sqrt(2)).astype(float)
    df['distance to fault'] = dist
    df['fault indicator'] = (dist < 500)*1
    for well in unique_list:
        df.loc[df["Well_ID"] == well, "Avg_perm"] =(df.loc[(df["Well_ID"] == well) & (~ (np.isnan(df["Perm"]))), "Perm"]).mean()
        df.loc[df["Well_ID"] == well, "thickness"] = df.loc[df["Well_ID"] == well, "Depth"].max() - df.loc[df["Well_ID"] == well, "Depth"].min()
        df.loc[df["Well_ID"] == well, "bottom to woc"] =  woc - df.loc[df["Well_ID"] == well, "Depth"].max() 
        df.loc[df["Well_ID"] == well, "sand prp."] = df.loc[(df["Well_ID"] == well) & (df["Facies"] == "Sandstone")].shape[0]\
        /df.loc[(df["Well_ID"] == well)].shape[0]
        df.loc[df["Well_ID"] == well, "sandshaly prp."] = \
        df.loc[(df["Well_ID"] == well) & ((df["Facies"] == "Sandstone")|(df["Facies"] == "Shaly sandstone")) ].shape[0]\
        /df.loc[(df["Well_ID"] == well)].shape[0]
    df["k/log_mindist"] = df["Avg_perm"] / df["log_mindist"]
    if avg:
        # return the average properties
        df.loc[df['Y'] >= (-df['X'] + 11750), "location"] = 0
        df.loc[df['Y'] < (-df['X'] + 11750), "location"] = 1
        df['location'] = df['location'].astype('int')
        df = df.groupby(["Well_ID"], sort=False, as_index=False).mean()

    return df
 

## Merge static properties with production

Next, we take an extra step to merge the two dataframes we have for the producing wells `df_p` and their corresponding production data `df_production`. 

In [4]:
def merge(df, df_production):
    df_production['total1'] = df_production['Cumwater1'] + df_production['Cumoil1']
    df_production['total2'] = df_production['Cumwater2'] + df_production['Cumoil2']
    df_production['total3'] = df_production['Cumwater3'] + df_production['Cumoil3']  

    df_production['prod_o1'] = df_production['Cumoil1']
    df_production['prod_o2'] = df_production['Cumoil2'] - df_production['Cumoil1']
    df_production['prod_o3'] = df_production['Cumoil3'] - df_production['Cumoil2']

    df_production['prod_w1'] = df_production['Cumwater1']
    df_production['prod_w2'] = df_production['Cumwater2'] - df_production['Cumwater1']
    df_production['prod_w3'] = df_production['Cumwater3'] - df_production['Cumwater2']

    df_production['oil12'] = (df_production['prod_o2']- df_production['prod_o1'])/ df_production['prod_o1']
    df_production['oil23'] = (df_production['prod_o3']- df_production['prod_o2'])/ df_production['prod_o2']

    df_production['water12'] = (df_production['prod_w2']- df_production['prod_w1'])/ df_production['prod_w1']
    df_production['water23'] = (df_production['prod_w3']- df_production['prod_w2'])/ df_production['prod_w2']


    df_production['wcut1'] = df_production['Cumwater1'] / (df_production['Cumoil1'] + df_production['Cumwater1'])
    df_production['wcut2'] = (df_production['Cumwater2'] - df_production['Cumwater1']) / (df_production['Cumoil2'] - df_production['Cumoil1'] + df_production['Cumwater2'] - df_production['Cumwater1'])
    df_production['wcut3'] = (df_production['Cumwater3'] - df_production['Cumwater2'])/ (df_production['Cumoil3'] - df_production['Cumoil2'] + df_production['Cumwater3'] - df_production['Cumwater2'])

    df_production['wcutincrease1'] = (df_production['wcut2']- df_production['wcut1'])/ (df_production['wcut1'])
    df_production['wcutincrease2'] = (df_production['wcut3']- df_production['wcut2']) / (df_production['wcut2'])
    
    df_producers_average = df.groupby(["Well_ID"], sort=False, as_index=False).mean()
    df_producers_average.reset_index(drop=True, inplace=True)
    df_production.reset_index(drop=True, inplace=True)
    df_merged = pd.concat([df_producers_average, df_production.iloc[:,1:]], axis=1)
    
    return df_merged

## Estimate Pseudo drainage areas based on Voronoi diagram

One of the features we explored in our implementation is the drainage area around well. We construct Voronoi diagram on the wells of interest. From the diagram, we obtain the areas for the Voronoi regions and assign it to each well as a proxy for its drainage area. 

In [5]:
def calculate_voronoi_areas(df, plot=False):
    
    from scipy.spatial import Voronoi, voronoi_plot_2d
    
    points = np.stack([df["X"], df["Y"]]).T
    vor = Voronoi(points)
    if plot:
        fig = voronoi_plot_2d(vor)
        plt.gca().invert_yaxis()
        plt.show()
        
    def PolyArea(x,y):
        return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
    
    regions = vor.regions
    vertices = vor.vertices
    Areas = []
    point_region = vor.point_region
    for i in range(points.shape[0]):
        region = regions[point_region[i]]
        Areas.append(PolyArea(vertices[region,0], vertices[region, 1])/1000)
    Areas = np.array(Areas)
    max_area = Areas.max()
    np.count_nonzero(Areas== 0)
    Areas[np.where(Areas>=2000)] = 2000
    df['area_poly'] = Areas
    
    return df

## Estimate pressure drop

Below we define two functions to calculate estimates for the pressure drop. The first one `compute_dp_map` computes the pressure drop all over the grid. The second one `compute_dp_map` computes the pressure drop at an individual well location. The pessure drop at any location is the superposition of the drop caused by adjacent wells. i.e., 

$$ \Delta P = \sum_{i}^{N} \Delta P_i$$
where, 
$$ \Delta P_i = \frac{-70.6qB\mu}{kh} Ei \left(\frac{-984 \phi \mu c_t r_i ^2}{kt}\right) $$

for all wells $i\in[N]$. $r_i$ is the Euclidean distance between a contributing well $i$ and the point of interest. 

Similarly, the pressure drop at a  well location can be estimated in a similar manner and adding a term that accounts for the drop due to its own depletion: 
$$ \Delta P_A = \Delta P_A + \sum_{i}^{N} \Delta P_i$$
where, 

$$ \Delta P_A = \frac{-70.6qB\mu}{kh} \ln \left(\frac{-1688 \phi \mu c_t r_w ^2}{kt}\right) $$
$$ \Delta P_i = \frac{-70.6q_iB\mu}{kh} Ei \left(\frac{-984 \phi \mu c_t r_{i-A} ^2}{kt}\right) $$

In [6]:
from scipy.spatial.distance import cdist
from scipy.special import expi

def compute_dp_map(grid, wells, poros, perms, prods_oil, prods_water, t):
    beta_o = 1.5
    beta_w = 1.05
    mu_o = 20.
    mu_w = 1
    h = 10 * 3.28084
    c_t = 1e-6
    distances = cdist(grid, wells)
    distances[np.where(distances< 10)] = 10
    n = grid.shape[0]
    delta_p = np.zeros(n)
    
    delta_p += np.sum(-70.6 * prods_oil *beta_o * mu_o / (perms * h) * expi(-984 * 10.7639 * poros * mu_o * c_t * distances **2 / (perms * t)), axis =1)
    delta_p += np.sum(-70.6 * prods_water *beta_w * mu_w / (perms * h) * expi(-984 * 10.7639 * poros * mu_w * c_t * distances **2 / (perms * t)), axis =1)

    return delta_p

def compute_dp_well(well, wells,por, poros, perm,  perms, prod_oil, prods_oil, prod_water, prods_water, t):
    rw = 0.2
    beta_o = 1.5
    beta_w = 1.05
    mu_o = 20.
    mu_w = 1
    h = 10 * 3.28084
    c_t = 1e-6
    distances = cdist(well, wells)

    delta_p = -70.6 * prod_oil *beta_o * mu_o / (perm * h) * np.log(1688 * mu_o *por *c_t * rw **2 / (perm * t))
    delta_p += -70.6 * prod_water *beta_w * mu_w / (perm * h) * np.log(1688 * mu_w *por *c_t * rw **2 / (perm * t))

    for i in range(len(wells)):
        delta_p += -70.6 * prods_oil[i] *beta_o * mu_o / (perms[i] * h) * expi(-984 * 10.7639 * poros[i] * mu_o * c_t * distances.T[i] **2 / (perms[i] * t))
        delta_p += -70.6 * prods_water[i] *beta_w * mu_w / (perms[i] * h) * expi(-984 * 10.7639 * poros[i] * mu_w * c_t * distances.T[i] **2 / (perms[i] * t))
    return delta_p



In [7]:
def calculate_dp_df(df):
    from scipy.spatial.distance import cdist
    x = np.linspace(25,10000 -25, 200)
    y = x
    X, Y = np.meshgrid(x, y, indexing='ij' )
    xy = np.array([df['X'].values, df['Y'].values]).T
    prods_oil1 = df['Cumoil1'].values
    prods_oil2 = df['Cumoil2'].values - df['Cumoil1'].values
    prods_oil3 = df['Cumoil3'].values - df['Cumoil2'].values

    prods_water1 = df['Cumwater1'].values
    prods_water2 = df['Cumwater2'].values - df['Cumwater1'].values
    prods_water3 = df['Cumwater3'].values - df['Cumwater2'].values

    poros = df['Porosity'].values
    perms = df['Avg_perm'].values

    positions = np.array((X,Y)).T

    positions=positions.reshape([40000, 2])


    distances = cdist(positions, xy)
    distances.shape
    n = positions.shape[0]
    delta_p = np.zeros(n)


    avg_por = 0.128
    avg_perm = 80.


    dp_1 = compute_dp_map(positions, xy, avg_por, avg_perm, prods_oil1/365, prods_water1/365,1*365*24)
    dp_2 = compute_dp_map(positions, xy, avg_por, avg_perm, prods_oil2/365, prods_water2/365, 1*365*24) + dp_1
    dp_3 = compute_dp_map(positions, xy, avg_por, avg_perm, prods_oil3/365, prods_water3/365, 1*365*24) + dp_2

    dp_1 = dp_1.reshape([200,200])
    dp_2 = dp_2.reshape([200,200])
    dp_3 = dp_3.reshape([200,200])


    np.save("./engineered_maps/dp_1", dp_1)
    np.save("./engineered_maps/dp_2", dp_2)
    np.save("./engineered_maps/dp_3", dp_3)


    for well in df["Well_ID"].unique():
            well_coords = np.array([df.loc[df["Well_ID"] == well, "X"].values, df.loc[df["Well_ID"] == well, "Y"].values]).T
            wells_coords = np.array([df.loc[df["Well_ID"] != well, "X"].values, df.loc[df["Well_ID"] != well, "Y"].values]).T

            prod_oil_1 = df.loc[df["Well_ID"] == well, "Cumoil1"].values
            prod_oil_2 = df.loc[df["Well_ID"] == well, "Cumoil2"].values - prod_oil_1
            prod_oil_3 = df.loc[df["Well_ID"] == well, "Cumoil3"].values - prod_oil_2

            prod_water_1 = df.loc[df["Well_ID"] == well, "Cumwater1"].values
            prod_water_2 = df.loc[df["Well_ID"] == well, "Cumwater1"].values - prod_water_1
            prod_water_3 = df.loc[df["Well_ID"] == well, "Cumwater1"].values - prod_water_2

            prods_oil_1 = df.loc[df["Well_ID"] != well, "Cumoil1"].values
            prods_oil_2 = df.loc[df["Well_ID"] != well, "Cumoil2"].values - prods_oil_1
            prods_oil_3 = df.loc[df["Well_ID"] != well, "Cumoil3"].values - prods_oil_2

            prods_water_1 = df.loc[df["Well_ID"] != well, "Cumwater1"].values
            prods_water_2 = df.loc[df["Well_ID"] != well, "Cumwater1"].values - prods_water_1
            prods_water_3 = df.loc[df["Well_ID"] != well, "Cumwater1"].values - prods_water_2

            por = df.loc[df["Well_ID"] == well, "Porosity"].values
            perm = df.loc[df["Well_ID"] == well, "Avg_perm"].values
            poros = df.loc[df["Well_ID"] != well, "Porosity"].values
            perms = df.loc[df["Well_ID"] != well, "Avg_perm"].values

            avg_perms = np.ones_like(perms) * avg_perm
            avg_pors = np.ones_like(poros) * avg_por

            df.loc[df["Well_ID"] == well, "dp_1"] = compute_dp_well(well_coords, wells_coords, avg_por, avg_pors, avg_perm, avg_perms, prod_oil_1/365,prods_oil_1/365, prod_water_1/365,prods_water_1/365,1*365*24)
            df.loc[df["Well_ID"] == well, "dp_2"] = compute_dp_well(well_coords, wells_coords, avg_por, avg_pors, avg_perm, avg_perms, prod_oil_1/365,prods_oil_1/365, prod_water_1/365,prods_water_1/365,1*365*24)
            df.loc[df["Well_ID"] == well, "dp_3"] = compute_dp_well(well_coords, wells_coords, avg_por, avg_pors, avg_perm, avg_perms, prod_oil_1/365,prods_oil_1/365, prod_water_1/365,prods_water_1/365,1*365*24)
    return df

## Estimate WOC rise

Now, we fill in our estimations for the WOC rise at the well locations, depending on the maps we generated in `Volumetric Caclulations.ipynp`. We simply interpolate at the well locations. 

In [8]:
def calculate_woc_rise_df(df):
    df["woc rise 0"] = 0
    df["woc rise 1"] = woc_rise_1[df['X'].astype('int')//50, df['Y'].astype('int')//50] 
    df["woc rise 2"] = woc_rise_2[df['X'].astype('int')//50, df['Y'].astype('int')//50] 
    df["woc rise 3"] = woc_rise_3[df['X'].astype('int')//50, df['Y'].astype('int')//50] 
    df["bottom to woc 1"] = df["bottom to woc"] - woc_rise_1[df['X'].astype('int')//50, df['Y'].astype('int')//50]
    df["bottom to woc 2"] = df["bottom to woc"] - woc_rise_2[df['X'].astype('int')//50, df['Y'].astype('int')//50]
    df["bottom to woc 3"] = df["bottom to woc"] - woc_rise_3[df['X'].astype('int')//50, df['Y'].astype('int')//50]
    return df

## Write files to machine

Save the final preprocessed datasets 

In [9]:
N = 100

for i in range(N):
    df_p = pd.read_csv('./imputed_data/df_p_{0:03d}.csv'.format(i))
    df_n = pd.read_csv('./imputed_data/df_n_{0:03d}.csv'.format(i))
    df_p.columns = ['Well_ID','X','Y','Depth','Porosity','Perm','AI', 'Facies', 'Density','Comp_vel','E','Vs','G', 'layer', 'lith_id']
    df_n.columns = ['Well_ID','X','Y','Depth','Porosity','Perm','AI', 'Facies', 'Density','Comp_vel','E','Vs','G', 'layer', 'lith_id']
    df_p = derive_static_prop(df_p)
    df_n = derive_static_prop(df_n)
    df_p = calculate_voronoi_areas(df_p)
    df_n = calculate_voronoi_areas(df_n)
    df_p = calculate_woc_rise_df(df_p)
    df_n = calculate_woc_rise_df(df_n)
    df_merged = merge(df_p, df_production)
    df_merged = calculate_dp_df(df_merged)
    df_n.to_csv('./preprocessed_data/df_n_preprocessed_{0:03d}.csv'.format(i), index=False)
    df_p.to_csv('./preprocessed_data/df_p_preprocessed_{0:03d}.csv'.format(i), index=False)
    df_merged.to_csv('./preprocessed_data/df_merged_{0:03d}.csv'.format(i), index=False)
    